In [12]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
import psycopg2
import pandas as pd
import os, json
from sqlalchemy import Table, Column, Integer, Unicode, MetaData, create_engine
from sqlalchemy.orm import mapper, create_session
from sqlalchemy_utils import database_exists

In [13]:
dbname = 'restaurants'
username = 'andylane'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://andylane@localhost/restaurants


In [14]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [9]:
def flatten_db(dictout):
    for index, item in enumerate(dictout["restaurants"]):
        for key, value in item["restaurant"].iteritems():
            if type(value) == dict:
                testme = (key,value)
                dictout["restaurants"][index]["restaurant"][str(key)] = json.dumps(value)
    return dictout

In [10]:
menus = []
flatdictout = []
for filename in os.listdir("./output"):
    dictout=[]
    with open(str("./output/" + filename), "r") as file:
        dictout.append(flatten_db(json.load(file)))
    for item in dictout:
        flatdictout.extend(item["restaurants"])
    #         for item in dictout["restaurants"]:
    #             menus.append(item["restaurant"]["menu_url"])
pd.DataFrame.from_dict([item["restaurant"] for item in flatdictout]).to_sql('restaurants', engine, if_exists='replace')

In [118]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

In [115]:
columnheaders = list(dictout["restaurants"][0]["restaurant"].iterkeys())

In [116]:
columnheaders

[u'cuisines',
 u'establishment_types',
 u'currency',
 u'featured_image',
 u'average_cost_for_two',
 u'deeplink',
 u'id',
 u'thumb',
 u'is_delivering_now',
 u'menu_url',
 u'photos_url',
 u'user_rating',
 u'offers',
 u'location',
 u'has_table_booking',
 u'events_url',
 u'R',
 u'price_range',
 u'apikey',
 u'name',
 u'has_online_delivery',
 u'url']

In [120]:
class Restaurant(object):
    pass

metadata = MetaData(bind=engine)

t = Table('restaurants', metadata, Column('id', Integer, primary_key=True),
    *(Column(wordCol, Unicode(255)) for wordCol in columnheaders))
metadata.create_all()
mapper(Restaurant, t)
session = create_session(bind=engine, autocommit=False, autoflush=True)

ArgumentError: Trying to redefine primary-key column 'id' as a non-primary-key column on table 'restaurants'